In [1716]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sklearn

In [1717]:
def to_hours(time):
    return (time // 3600) + 6


def clean_sales_data():
    df = pd.read_csv("../data/master_data/sales_data.csv")
    sales_df = df[["token_id", "sold_at", "price"]]
    sales_df = (
        sales_df.rename(columns={"token_id": "horse_id", "sold_at": "time_sold"})
        .sort_values("horse_id")
        .reset_index(drop=True)
    )
    sales_df["timestamp"] = sales_df["time_sold"].apply(to_hours)

    return sales_df

## Data merging code


In [1718]:
filenames = [
    "sales_and_stats_dennis_pt2.csv",
    "sales_and_stats_dennis_pt_1_8k .csv",
    "sales_and_stats_harper.csv",
    "sales_and_stats_jason.csv",
]

meta_data_races = pd.read_csv("../sales_and_stats_oliver.csv")

for file in filenames:
    new_df = pd.read_csv(f"../{file}")
    meta_data_races = pd.concat([meta_data_races, new_df], axis=0)

print(meta_data_races.shape)
meta_data_attributes = pd.read_csv("../data/master_data/backup_meta_db.csv")

# print(f"columns of meta_data_races = \n {meta_data_races.columns}")
# print(f"columns of meta_data_attributes = \n {meta_data_attributes.columns}")

merged = meta_data_attributes.copy()

meta_merged = merged.merge(meta_data_races, on="horse_id", how="outer").fillna(0)
meta_merged = meta_merged.drop(
    columns=[
        "Unnamed: 0_y",
        "Unnamed: 0_x",
    ]
)
meta_merged.columns

(51023, 11)


Index(['currency', 'price', 'sold_at', 'horse_id', 'genotype', 'bloodline',
       'breed_type', 'color', 'birthday', 'super_coat', 'mother', 'father',
       'horse_type', 'free_win_rate', 'paid_win_rate', 'place', 'total_paid',
       'win_rate', 'overall.first', 'overall.races', 'overall.second',
       'overall.third'],
      dtype='object')

In [1719]:
# replace complex token names with easier to understand names and drop rows paid for with zed_tokens

meta_merged.replace(
    to_replace="0x5ec03c1f7fa7ff05ec476d19e34a22eddb48acdc",
    value="zed_token",
    inplace=True,
)
meta_merged.replace(
    to_replace="0x7ceb23fd6bc0add59e62ac25578270cff1b9f619", value="eth", inplace=True
)

meta_merged = meta_merged[meta_merged["currency"] != "zed_token"]
meta_merged.currency.value_counts()

# drop currency column now that price is just eth

meta_merged = meta_merged.drop(columns=["currency"])

### convert sales times to match eth price times


In [1720]:
meta_sales = (
    meta_merged.rename(columns={"sold_at": "time_sold"})
    .sort_values("horse_id")
    .reset_index(drop=True)
)
meta_sales["timestamp"] = meta_sales["time_sold"].apply(to_hours)

In [1721]:
# sales_data = clean_sales_data()

In [1722]:
eth_prices = pd.read_csv("../data/master_data/eth_price_history.csv")
eth_prices.tail()

,Unnamed: 0,timestamp,eth_price
9091,9091,464923,1533.01
9092,9092,464924,1529.65
9093,9093,464925,1535.38
9094,9094,464926,1538.76
9095,9095,464927,1550.97


In [1723]:
eth_merge = pd.merge(
    meta_sales, eth_prices, left_on="timestamp", right_on="timestamp", how="left"
)
eth_merge["converted_price"] = eth_merge["price"] * eth_merge["eth_price"]
clean_market_data = eth_merge.drop(
    columns=["Unnamed: 0", "price", "timestamp", "eth_price"]
)


def to_day(unix_time):
    return datetime.date.fromtimestamp(unix_time)


clean_market_data["day_sold"] = clean_market_data["time_sold"].apply(to_day)
clean_market_data.to_csv("mega_master_db.csv")
clean_market_data

,time_sold,horse_id,genotype,bloodline,breed_type,color,birthday,super_coat,mother,father,...,paid_win_rate,place,total_paid,win_rate,overall.first,overall.races,overall.second,overall.third,converted_price,day_sold
0,1.650125e+09,27,Z1,Nakamoto,genesis,Absolute Zero,2019-01-31T03:48:43,False,0.0,0.0,...,8.20,39.37,4.786438,12.41,226,1821,254,237,60665.000,2022-04-16
1,1.657313e+09,68,Z1,Nakamoto,genesis,Slate Gray,2019-02-07T09:20:41,False,0.0,0.0,...,0.00,57.89,0.039840,13.16,5,38,5,12,13934.205,2022-07-08
2,1.657313e+09,75,Z1,Nakamoto,genesis,Slate Gray,2019-02-07T11:15:31,False,0.0,0.0,...,0.00,55.00,0.002400,20.00,4,20,5,2,9693.360,2022-07-08
3,1.657313e+09,89,Z1,Nakamoto,genesis,Slate Gray,2019-02-15T19:45:15,False,0.0,0.0,...,0.00,56.52,0.009720,21.74,5,23,3,5,9693.360,2022-07-08
4,1.663112e+09,92,Z1,Nakamoto,genesis,Champagne Papi,2019-02-16T13:09:35,False,0.0,0.0,...,13.04,38.04,0.160059,10.14,28,276,37,40,10943.775,2022-09-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74779,0.000000e+00,520344,0,0,0,0,0,0,0.0,0.0,...,11.11,20.69,0.006528,6.90,2,29,4,0,NaN,1969-12-31
74780,0.000000e+00,520345,0,0,0,0,0,0,0.0,0.0,...,0.00,27.12,0.000000,10.17,6,59,7,3,NaN,1969-12-31
74781,0.000000e+00,520346,0,0,0,0,0,0,0.0,0.0,...,14.29,30.23,0.016128,16.28,7,43,3,3,NaN,1969-12-31
74782,0.000000e+00,520404,0,0,0,0,0,0,0.0,0.0,...,10.00,25.00,0.083148,8.33,4,48,4,4,NaN,1969-12-31


### Deal with outliers


In [1724]:
# Calculate the z-score for each data point
clean_market_data["z_score"] = np.abs(
    (
        clean_market_data["converted_price"]
        - np.mean(clean_market_data["converted_price"])
    )
    / np.std(clean_market_data["converted_price"])
)

clean_market_data
market_data_no_outliers = clean_market_data[clean_market_data["z_score"] < 3]
print(
    clean_market_data.shape,
    market_data_no_outliers.shape,
    f"=> {74911 - 74820} data points removed",
)

(74784, 23) (74389, 23) => 91 data points removed


In [1725]:
market_data_no_outliers

,time_sold,horse_id,genotype,bloodline,breed_type,color,birthday,super_coat,mother,father,...,place,total_paid,win_rate,overall.first,overall.races,overall.second,overall.third,converted_price,day_sold,z_score
15,1.671901e+09,270,Z1,Nakamoto,genesis,Rosy Brown,2019-03-02T09:29:40,False,0.0,0.0,...,24.63,1.760085,3.59,34,946,79,120,3046.15000,2022-12-24,2.345064
24,1.663184e+09,501,Z2,Nakamoto,genesis,Electric Violet,2019-03-15T12:08:00,False,0.0,0.0,...,22.87,0.091390,5.38,12,223,16,23,3093.96000,2022-09-14,2.384025
25,1.663610e+09,501,Z2,Nakamoto,genesis,Electric Violet,2019-03-15T12:08:00,False,0.0,0.0,...,22.87,0.091390,5.38,12,223,16,23,1801.86500,2022-09-19,1.331101
28,1.673439e+09,545,Z2,Nakamoto,genesis,Papaya Whip,2019-03-17T07:05:34,False,0.0,0.0,...,20.74,0.733313,2.82,23,815,62,84,1792.11150,2023-01-11,1.323153
37,1.655247e+09,644,Z2,Nakamoto,genesis,Brown Sugar,2019-03-20T11:32:57,False,0.0,0.0,...,24.37,0.143701,4.30,12,279,25,31,3337.74900,2022-06-14,2.582687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74761,1.673462e+09,519821,Z268,Buterin,cross,Cosmic Cobalt,2023-01-07T19:47:54,False,368861.0,509644.0,...,0.00,0.000000,0.00,0,8,0,0,1.38913,2023-01-11,0.136101
74762,1.673563e+09,519849,Z17,Buterin,elite,Oval Orchid,2023-01-07T22:27:17,False,473513.0,489756.0,...,18.18,0.000000,9.09,1,11,1,0,23.34585,2023-01-12,0.118208
74763,1.673312e+09,519892,Z12,Finney,exclusive,Midnight Black,2023-01-08T03:01:52,False,443585.0,2605.0,...,29.91,0.229800,13.08,14,107,11,7,66.24200,2023-01-09,0.083252
74764,1.673477e+09,519901,Z6,Szabo,exclusive,Dark Moss,2023-01-08T04:15:02,False,513329.0,24283.0,...,23.19,0.056379,8.70,6,69,6,4,11.20592,2023-01-11,0.128101
